# Loading Data

In [1]:
train_csv_path = '../input/toy-dataset-for-regression-and-uq/Data3_Train.csv'
test_csv_path = '../input/toy-dataset-for-regression-and-uq/Data3_Test.csv'
val_csv_path = '../input/toy-dataset-for-regression-and-uq/Data3_Val.csv'



import pandas as pd
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
train_df.head()

,Input1,Input2,Input3,Output
0,2.659116,4.673695,1.231898,-1.010633
1,6.172297,0.308095,6.197296,0.140169
2,1.620350,3.752477,3.010036,0.014108
3,6.269573,3.173038,6.019555,-0.016590
4,3.946378,3.264015,2.245954,1.222183


# Data Arranging and Normalizing

In [2]:
input_=[]
output_=[]
for row in train_df.iloc:
    input_.append((row[0:len(row)-1]).astype(float))
    output_.append(row[-1])
    
i_val=[]
o_val=[]
for row in val_df.iloc:
    i_val.append((row[0:len(row)-1]).astype(float))
    o_val.append(row[-1])
    
num_input = len(row)-1;
print(num_input)

####################################################
#
# This code is written with the help of the demo:
# https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379
#
####################################################
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import imageio


torch.manual_seed(1)    # reproducible

x = torch.tensor(input_).float()  
y = torch.tensor(output_).float()   
new_shape = (len(y), 1)
y = y.view(new_shape)


i_val = torch.tensor(i_val).float()  
o_val = torch.tensor(o_val).float()   
new_shape = (len(o_val), 1)
o_val = o_val.view(new_shape)



max_y = torch.max(y[:,0])
min_y =torch.min(y[:,0])

max_x = torch.max(x,dim=0)
min_x = torch.min(x,dim=0)

print(max_y, min_y, max_x.values, min_x.values)

range_y = max_y - min_y
range_x = max_x.values - min_x.values

print(range_x, range_y)

    #Normalizing
x = (x - min_x.values)/range_x
y = (y - min_y)/range_y

    #Normalizing
i_val = (i_val - min_x.values)/range_x
o_val = (o_val - min_y)/range_y

3
tensor(1.4626) tensor(-1.4431) tensor([6.2822, 6.2817, 6.2823]) tensor([4.0563e-04, 6.2234e-05, 9.0123e-04])
tensor([6.2818, 6.2816, 6.2814]) tensor(2.9057)


# Prediction NN Training

In [3]:
# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)
 
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x

# use the same net as before      
net = Net(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net)  # net architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


minimum_train_loss = 1e5
minimum_val_loss = 1e5
EPOCH = 600

# start training
for epoch in range(EPOCH):
  
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net
    
    if epoch%200 == 199:
      prediction = net_opt(i_val)
      loss_val = loss_func(prediction, o_val)
      if loss_val<minimum_val_loss:
        minimum_val_loss = loss_val
        net_opt_val = net_opt
      print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))

Net(
  (hidden): Linear(in_features=3, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Epoch [200/600], Loss: 0.0133, Minimum Loss 0.012997, Val Loss 0.015020  
Epoch [400/600], Loss: 0.0060, Minimum Loss 0.005954, Val Loss 0.006948  
Epoch [600/600], Loss: 0.0054, Minimum Loss 0.004941, Val Loss 0.005750  


# Computing Test Loss

In [4]:
test_df = pd.read_csv(test_csv_path)

i_test=[]
o_test=[]
for row in val_df.iloc:
    i_test.append((row[0:len(row)-1]).astype(float))
    o_test.append(row[-1])


   
i_test, o_test = Variable(torch.tensor(i_test)).float(), Variable(torch.tensor(o_test).float())
new_shape = (len(o_test), 1)
o_test = o_test.view(new_shape)

    #Normalizing
i_test = (i_test - min_x.values)/range_x
o_test = (o_test - min_y)/range_y

prediction = net_opt_val(i_test)
loss_test = loss_func(prediction, o_test)

print("Normalized Test Loss",loss_test.detach().numpy())

loss_test = loss_test*range_y*range_y # As the loss function returns MSE

print("Test Loss",loss_test.detach().numpy())

Normalized Test Loss 0.005750177
Test Loss 0.048549186


# Minimim deviation based similarity finding for Uncertainty Bounds

   Aim: To achieve all indexes of similar samples

   Steps: 

1. Itertion over all Input combinations.

2. Find all deviation by indexes 
   (sensitivity can be considered during the consideration of deviation)

3. Save top 100 similar events


In [5]:
import tensorflow as tf
import pickle as pkl
import time


Similar_event_count = 100
sensitivity_consideration = 1
max_dev_index = np.zeros(shape=(len(x),2))
Similarity = np.zeros(shape=(len(x),Similar_event_count+1))
sensitivity = torch.ones((len(max_x.values)), dtype=torch.float64) # just initializing a variable of that size
iter_debug = 0
since = time.time()

for iter1 in range(len(x)):
    current_input = x[iter1,:];
    
    if sensitivity_consideration == 1:
        current_output = net_opt_val(current_input);
        
       # Determining Sensitivity near input[iter1]
        for iter2 in range(len(current_input)): 
            #print('Before Change',current_input)
            current_input[iter2] = current_input[iter2] + range_x[iter2]/1e4;
            #print('After Change',current_input)
            sensitivity[iter2] = torch.abs(current_output - net_opt_val(current_input));
            current_input[iter2] = current_input[iter2] - range_x[iter2]/1e4;
        
        sensitivity = sensitivity/torch.max(sensitivity); 
        #The highest sensitivity is considered one
    
    for iter2 in range(len(x)):
        max_dev_index[iter2][0] = torch.max(sensitivity*torch.abs(current_input-x[iter2,:])/range_x).detach().numpy()
        max_dev_index[iter2][1] = iter2
        
    sort_dev_index = max_dev_index[np.argsort(max_dev_index[:, 0])]
    Similarity_threshold = sort_dev_index[Similar_event_count][0]
    matched_indexes = sort_dev_index[0:Similar_event_count,1]
    
    Similarity[iter1,0] = Similarity_threshold
    Similarity[iter1,1:(Similar_event_count+1)] = matched_indexes
    
    if iter1%1000==100:   
        print(iter1/len(x)*100, '%' '  Complete')
        time_elapsed = time.time() - since
        # Observing progres in Command Window
        # May Take about hours to find all similar events for entire training data

print('100% Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    

fileName = './' + 'Similarity'
fileObject = open(fileName, 'wb')

pkl.dump(Similarity, fileObject)
fileObject.close()



2.0 %  Complete
22.0 %  Complete
42.0 %  Complete
62.0 %  Complete
82.0 %  Complete
100% Complete in 18m 44s


In [6]:
#Scripts for loading
#ileObject2 = open(fileName, 'wb')
#Test_array = pkl.load(fileObject2)
#fileObject2.close()